ТИТАНИК
======
Имеются данные о пассажирах Титаника. 

В результате работы необходимо построить классификатор, который по данным о пассажире будет предсказывать исход: выжил пассажир или нет.

## Часть 1. Работа с признаками.

Мы познакомимся с первичной обработкой данных, выявлением признаков для классификации, а также построим модели при помощи разных алгоритмов классификации и сравним результаты.

Для дальнейшей работы с данными подключим необходимые библиотеки:
- pandas - анализ данных
- numpy - работа с массивами
- pyplot - отображение данных
- seaborn - отображение данных 2
- re - регулярные выражения
- sklearn - классификаторы/кросс-валидатор/нормализатор/оценка признаков

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cross_validation import StratifiedKFold, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn import metrics

Далее необходимо загрузить в память имеющиеся данные: обучающую выборку (training set) и тестовую. Эти процедуры производятся при помощи библиотеки pandas:

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

**Задание.** Определите обеих размер выборок, например, при помощи встроенной функции len.

In [ ]:
# Место для вашего кода

Задолго до того, как мы приступим к построению моделей, необходимо сделать предварительный анализ данных. Да и вообще, понять с чем придется работать.

Для этого нужно "посмотреть" на данные и сделать некоторые выводы. Чтобы вывести данные в корректной табличной форме в библиотеке pandas есть метод head(n), который выводит первые n строк данных.

**Задание.** Посмотрите на первые строки выборки train_data.

In [ ]:
# Место для вашего кода

Проанализируем поля данных. У нас есть:
- survival - 	Пассажир выжил	0 = нет, 1 = да
- pclass - Класс билета(каюты)	1 = 1ый, 2 = 2ой, 3 = 3ий
- sex - Пол	
- Age - Возраст (полных лет)	
- sibsp	- братьев/сестер/супругов на борту	
- parch	- родителей/детей на борту	
- ticket - номер билета	
- fare	- тариф	
- cabin	- номер каюты	
- embarked - порт посадки

**Задание.** Посмотрите внимательно на данные и попробуйте ответить на вопрос какие закономерности, или наоборот аномалии вы в них наблюдаете (5-7 мин). 

In [ ]:
# Место для вашего кода

Продолжим первичный анализ данных: полезно вычислить средние значения, знать минимальные и максимальные значения по каждому полю. 
Для того, чтобы вычислять эти характеристики, хорошо было бы иметь как можно больше данных. Можно заметить, что тренировочная и тестовая выборки имеют одну и ту же структуру. Поэтому, есть смысл их объединить в один большой массив данных и анализировать его. Так мы получим более точные средние значения, минимумы, максимумы и т.п.

Для того, чтобы объединить выборки в один массив можно использовать функцию из библиотеки panda concat.

**Задание.** Назовите новый массив all_data.

In [ ]:
# Место для вашего кода

Для вычисления средних значений, значений минимумов и максимумов, а также среднеквадратических отклонений и квантилей, удобно использовать функцию descibe. 

**Задание.** Примените её к созданному массиву all_data и проанализируйте результаты.

In [ ]:
# Место для вашего кода

На этом этапе уже можно сделать некоторые предположения о выживаемости, исходя из здравого смысла.

Скорее всего выживаемость пассажиров зависит в первую очередь от класса. То есть, "богатые" имели больше шансов выжить, чем "бедные", которые были расселены по каютам внизу корабля.
Кроме того, из истории происшествия известен факт, что при эвакуации старались спасать в большей степени женщин и детей.

Рассмотрим связь выживаемости и сочетания класса каюты + пол. Сделаем это при помощи метода библиотеки pandas groupby.

In [ ]:
train_data.groupby(["Pclass", "Sex"])["Survived"].value_counts(normalize=True)

**Задание.** Какие выводы можно сделать исходя из полученных данных?

Далее при помощи уже известного метода describe можно оценить числовые характеристики данных отдельно для мужчин и для женщин.
Но есть смысл рассматривать не все поля, а числовые и, вообще, имеющие для нас какой-то здравый смысл.

In [ ]:
describe_fields = ["Age", "Fare", "Pclass", "SibSp", "Parch"]

print("Мужчины")
print(train_data[train_data["Sex"] == "male"][describe_fields].describe(percentiles=None))

**Задание.** Выведите аналогичные данные для женщин.

In [ ]:
# Место для вашего кода

Когда первичный анализ данных проведен, можно подготавливать признаки для классификации.

Как мы заметили выше, у пассажиров могут быть заполнены не все поля. Оставлять в поле значение NaN нельзя, поэтому делаем важный шаг: 
** Вместо отсутствующих значений подставляем значение медианы, вычисленное по всему столбцу **
(Почему именно медиана, а не, к примеру, среднее арифметическое?).

Таким образом, мы "соберем" медианы по возрасту (ages) в специально созданный экземпаляр класса DataDigest. 

In [ ]:
class DataDigest:
    def __init__(self):
        self.ages = None
        self.fares = None
        self.titles = None
        self.cabins = None
        self.families = None
        self.tickets = None
        
data_digest = DataDigest()
data_digest.ages = train_data.groupby("Sex")["Age"].median()

**Задание.** Заполните поле fares аналогично тому как это сделано с полем ages

In [ ]:
# Место для вашего кода

Далее, попытаемся собрать новые признаки. Стратегия такая: лучше сделаем побольше признаков, а потом выясним какие соледует использовать. 

Какие признаки можно собрать ещё? Например, "вытащить" из поля с именем титулы (titles): мистер, мисс, миссис и т.д. В них содержится не только информация о половой принадлежности, но и о возрастной.

Можно собрать идентификаторы семей: фамилия + количество человек с такой фамилией на борту (families). 

Также, сделаем отдельные поля-признаки для кают (cabins) и билетов (tickets).

** Замечание! **

Обязательно нужно все текстовые поля перевести в числовой формат.

In [ ]:
def get_title(name):
    # извлекает титул из имени если он указан в его начале
    if pd.isnull(name):
        return "Null"

    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1).lower()
    else:
        return "None"

def get_family(row):
    # определяет принадлежность семье и ее размер
    last_name = row["Name"].split(",")[0]
    if last_name:
        family_size = 1 + row["Parch"] + row["SibSp"]
        if family_size > 3:
            return "{0}_{1}".format(last_name.lower(), family_size)
        else:
            return "nofamily"
    else:
        return "unknown"

# применим функцию get_title к ячейкам в поле Name, полученный список титулов сохраним
data_digest.titles = pd.Index(train_data["Name"].apply(get_title).unique())
# применим функцию get_family к строкам датафрейма, полученный список семей сохраним
data_digest.families = pd.Index(train_data.apply(get_family, axis=1).unique())
# все различные значения кабин тоже сохраним
data_digest.cabins = pd.Index(train_data["Cabin"].fillna("unknown").unique())
# все различные значения билетов тоже сохраним
data_digest.tickets = pd.Index(train_data["Ticket"].fillna("unknown").unique())

**Задание.** Выведите тип данных поля data_digest.titles и сами значения.

In [ ]:
# Место для вашего кода

Класс pandas.indexes.base.Index (или посто Index) позволяет получать по значению его позицию в списке с помощью функции get_loc. Возможность получать по значению его позицию может быть реализована и другими способами, например, функция index у обычного list.

Далее мы создадим сразу много числовых признаков (feature engineering) и разберем как у нас это получилось. Новые признаки:
- индекс каюты
- индекс палубы (вырезаны из исходных данных по каютам)
- индекс билета
- индекс титула (вырезаем из имени)
- индекс идентификатора семьи (фамилия + братья/сестры + родители/дети)


In [ ]:
def get_index(item, index):
    if pd.isnull(item):
        return -1

    try:
        return index.get_loc(item)
    except KeyError:
        return -1


def munge_data(data, digest):
    # Age - замена пропусков на медиану в зависимости от пола
    data["AgeF"] = data.apply(lambda r: digest.ages[r["Sex"]] if pd.isnull(r["Age"]) else r["Age"], axis=1)

    # Fare - замена пропусков на медиану в зависимости от класса
    data["FareF"] = data.apply(lambda r: digest.fares[r["Pclass"]] if pd.isnull(r["Fare"]) else r["Fare"], axis=1)

    # Gender - замена
    genders = {"male": 1, "female": 0}
    data["SexF"] = data["Sex"].apply(lambda s: genders.get(s))

    # Gender - расширение
    gender_dummies = pd.get_dummies(data["Sex"], prefix="SexD", dummy_na=False)
    data = pd.concat([data, gender_dummies], axis=1)

    # Embarkment - замена
    embarkments = {"U": 0, "S": 1, "C": 2, "Q": 3}
    data["EmbarkedF"] = data["Embarked"].fillna("U").apply(lambda e: embarkments.get(e))

    # Embarkment - расширение
    embarkment_dummies = pd.get_dummies(data["Embarked"], prefix="EmbarkedD", dummy_na=False)
    data = pd.concat([data, embarkment_dummies], axis=1)

    # Количество родственников на борту
    data["RelativesF"] = data["Parch"] + data["SibSp"]

    # Человек-одиночка?
    data["SingleF"] = data["RelativesF"].apply(lambda r: 1 if r == 0 else 0)

    # Deck - замена
    decks = {"U": 0, "A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "T": 8}
    data["DeckF"] = data["Cabin"].fillna("U").apply(lambda c: decks.get(c[0], -1))

    # Deck - расширение
    deck_dummies = pd.get_dummies(data["Cabin"].fillna("U").apply(lambda c: c[0]), prefix="DeckD", dummy_na=False)
    data = pd.concat([data, deck_dummies], axis=1)

    # Titles - расширение
    title_dummies = pd.get_dummies(data["Name"].apply(lambda n: get_title(n)), prefix="TitleD", dummy_na=False)
    data = pd.concat([data, title_dummies], axis=1)

    # замена текстов на индекс из соответствующего справочника или -1 если значения в справочнике нет (расширять не будем)
    data["CabinF"] = data["Cabin"].fillna("unknown").apply(lambda c: get_index(c, digest.cabins))

    data["TitleF"] = data["Name"].apply(lambda n: get_index(get_title(n), digest.titles))

    data["TicketF"] = data["Ticket"].apply(lambda t: get_index(t, digest.tickets))

    data["FamilyF"] = data.apply(lambda r: get_index(get_family(r), digest.families), axis=1)

    # для статистики
    age_bins = [0, 5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90]
    data["AgeR"] = pd.cut(data["Age"].fillna(-1), bins=age_bins).astype(object)

    return data


Cобираем новый набор данных.

In [ ]:
train_data_munged = munge_data(train_data, data_digest)
test_data_munged = munge_data(test_data, data_digest)
all_data_munged = pd.concat([train_data_munged, test_data_munged])

## Часть 2. Классификация.

Для решения этой задачи мы будем использовать несколько разных классификаторов и оценим насколько качественные результаты они покажут.
Так как разные алгоритмы работают с разными форматами данных, все данные необходимо **нормализовать**.
Массив всех известных нам признаков:

In [ ]:
features = ["Pclass",
              "AgeF",
              "TitleF",
              "TitleD_mr", "TitleD_mrs", "TitleD_miss", "TitleD_master", "TitleD_ms", 
              "TitleD_col", "TitleD_rev", "TitleD_dr",
              "CabinF",
              "DeckF",
              "DeckD_U", "DeckD_A", "DeckD_B", "DeckD_C", "DeckD_D", "DeckD_E", "DeckD_F", "DeckD_G",
              "FamilyF",
              "TicketF",
              "SexF",
              "SexD_male", "SexD_female",
              "EmbarkedF",
              "EmbarkedD_S", "EmbarkedD_C", "EmbarkedD_Q",
              "FareF",
              "SibSp", "Parch",
              "RelativesF",
              "SingleF"]

Для нормализации будем использовать стандартный нормализатор:

In [ ]:
scaler = StandardScaler()
scaler.fit(all_data_munged[features])

train_data_scaled = scaler.transform(train_data_munged[features])
test_data_scaled = scaler.transform(test_data_munged[features])

**Задание.** Найдите в Интернете информацию о StandardScaler. Ответьте на два вопроса:
- Какое преобразование над данными выполняет StandardScaler?
- Почему мы создаем только один экземпляр этого класса?

Как мы заметили ранее, выживаемость зависит от класса+возраста+пола. 
Мы можем проиллюстрировать это на графиках:

In [ ]:
sns.pairplot(train_data_munged, vars=["AgeF", "Pclass", "SexF"], hue="Survived", dropna=True)
sns.plt.show()

**Задание.** Проинтерпретируйте информацию на графиках.

**Задание.** Попробуйте при помощи методов groupby (как это было проделано ранее) вывести информацию о выживаемости пассажиров:
- относительно возраста
- относительно возраста + пола
- относительно класса + возраста

In [ ]:
#Место для вашего кода

При подготовке признаков мы, очевидно, сделали много лишних и малоинформативных.
Например, вряд ли порт посадки вносит хоть какой-то вклад в возможность выживания. Некоторые признаки дублируют друг друга (обе колонки, связанные с полом), а некоторые имеют сильную корреляцию (класс билета и стоимость билета).

Встроенными возможностями библиотеки scikit-learn можно оценить "важность" признаков. Проанализировав результаты, выбросим маловлияющие признаки и оставим для рассмотрения только самые ценные

In [ ]:
selector = SelectKBest(f_classif, k=5)
selector.fit(train_data_munged[features], train_data_munged["Survived"])

valuabilities = -np.log10(selector.pvalues_)

plt.bar(range(len(features)), valuabilities)
plt.xticks(range(len(features)), features, rotation='vertical')
plt.show()

В принципе, наши предположения подтвердились: очень важен пол, важен титул (но у него сильная корреляция с полом), весьма важен класс, и, почему-то, палуба F (видимо, дело в её расположении на корабле).

Теперь проделаем уже знакомые действия, собирая нормализованную выборку, но которая будет содержать только выбранные нами признаки.

**Задание.** В дальнейшем можно поэскпериментировать с набором признаков и посмотреть как будут меняться результаты классификации

In [ ]:
features = ["Pclass",
              "AgeF",
              "TitleF",
              "TitleD_mr", "TitleD_mrs", "TitleD_miss", "TitleD_master", "TitleD_ms", 
              "TitleD_rev", "TitleD_dr",
              "DeckF",
              "DeckD_U", "DeckD_A", "DeckD_C", "DeckD_D", "DeckD_E", "DeckD_F", "DeckD_G",
              "TicketF",
              "SexF",
              "SexD_male", "SexD_female",
              "EmbarkedF",
              "EmbarkedD_S", "EmbarkedD_C", "EmbarkedD_Q",
              "SibSp", "Parch",
              "SingleF"]
scaler = StandardScaler()
scaler.fit(all_data_munged[features])

train_data_scaled = scaler.transform(train_data_munged[features])
test_data_scaled = scaler.transform(test_data_munged[features])

Прежде чем перейти к классификации полезно разобраться с подходом называемым **кросс-валидация**. Его идея заключается в том, что мы делим тренировочную выборку на n частей, где все части кроме одной будут выступать в качестве тренировчной выборки, а оставшаяся - тестовой. Таким образом, алгоритм n раз "прогоняется" на тренировочных выборках, предсказывает на тестовых, и мы получаем значения accuracy - точности (отношение правильных оценок к общему числу). 

Прежде чем приступить к построению прогностической модели, разобьем выборку на 3 части.

In [ ]:
cv = StratifiedKFold(train_data["Survived"], n_folds=3, shuffle=True, random_state=1)

**Задание.** Ответьте на вопрос: что делает класс StratifiedKFold?

МЕТОД k БЛИЖАЙШИХ СОСЕДЕЙ (kNN)
====


In [ ]:
alg_ngbh = KNeighborsClassifier(n_neighbors=3)
scores = cross_val_score(alg_ngbh, train_data_scaled, train_data_munged["Survived"], cv=cv, n_jobs=-1)
print("All scores: {}".format(scores))
print("Accuracy: {} \nDisp: {}".format(scores.mean(), scores.std()))

Поэкспериментируйте со значением k. Что можно сказать по поводу полученных результатов?

ГРАДИЕНТНЫЙ СПУСК (Gradient descend)
====

In [ ]:
#Место для вашего кода

МЕТОД ОПОРНЫХ ВЕКТОРОВ (Support vector machine)
====

In [ ]:
#Место для вашего кода

НАИВНЫЙ БАЙЕСОВСКИЙ КЛАССИФИКАТОР (Naive Bayes classifier)
====

In [ ]:
#Место для вашего кода

ЛИНЕЙНАЯ РЕГРЕССИЯ (Linear regression)
===

Для линейной регрессии (поскольку она регрессия), нужно некоторое рубежное значение, которое позволит нам бинаризовать результат: 1 - выжил, 0 - умер.

** Замечание **

В предыдущих алгоритмах и далее n_jobs - параметр, который указывает число процессоров, которые будут задействованы в вычислениях.
Далее рекомендуется указывать значение n_jobs = 1 (вычислять на одном процессоре), т.к., почему-то, на некоторых компьютерах в иных случаях ноутбук зависает и отказывается считать.

In [ ]:
def linear_scorer(estimator, x, y):
    scorer_predictions = estimator.predict(x)

    scorer_predictions[scorer_predictions > 0.5] = 1
    scorer_predictions[scorer_predictions <= 0.5] = 0

    return metrics.accuracy_score(y, scorer_predictions)

In [ ]:
alg_lnr = LinearRegression()
scores = cross_val_score(alg_lnr, train_data_scaled, train_data_munged["Survived"], cv=cv, n_jobs=1,
                         scoring=linear_scorer)
print("Accuracy: {} \nDisp: {}".format(scores.mean(), scores.std()))

ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ (Logistic regression)
===

In [ ]:
#Место для вашего кода

СЛУЧАЙНЫЙ ЛЕС (Random forest)
===

Результат работы алгоритма случайного леса будет зависеть от входных параметров. Попытайтесь подобрать наиболее оптимальные с помощью grid_search. Эта функция позволяет перебрать декартово произведение множеств значений нескольких параметров (это называется grid) и для каждого варианта (ячейки в grid) вызвать функцию с этими параметрами.

**Комментарий.** Опытным путём установлено, что случайный лес даёт наилучшие результаты без переобучения среди приведенных алгоритмов при хорошо подобранных параметрах.

In [ ]:
#Место для вашего кода